In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

from annsa.template_sampling import *

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import model, training function 

In [4]:
from annsa.model_classes import (dnn_model_features,
                                 DNN,
                                 save_model,
                                 train_earlystop)

## Dataset Construction

#### Load dataset

In [5]:
background_dataset = pd.read_csv('../../source-interdiction/training_testing_data/background_template_dataset.csv')
source_dataset = pd.read_csv('../../source-interdiction/training_testing_data/shielded_templates_200kev_dataset.csv')

#### Dataset details

In [6]:
print('sourcedist: ' + str(sorted(set(source_dataset['sourcedist']))))
print('sourceheight: ' + str(sorted(set(source_dataset['sourceheight']))))
print('alum shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='alum']['shieldingdensity']))))
print('iron shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='iron']['shieldingdensity']))))
print('lead shieldingdensity: ' + str(sorted(set(source_dataset[source_dataset['shielding']=='lead']['shieldingdensity']))))
print('fwhm: ' + str(sorted(set(source_dataset['fwhm']))))

sourcedist: [50.0, 112.5, 175.0, 237.5, 300.0]
sourceheight: [50.0, 75.0, 100.0, 125.0, 150.0]
alum shieldingdensity: [1.82, 4.18, 7.49, 13.16]
iron shieldingdensity: [1.53, 3.5, 6.28, 11.02]
lead shieldingdensity: [0.22, 0.51, 0.92, 1.61]
fwhm: [6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0]


#### Customize dataset

In [7]:
source_dataset = source_dataset[source_dataset['fwhm']==7.5]
source_dataset = source_dataset[source_dataset['sourcedist']==175.0]
source_dataset = source_dataset[source_dataset['sourceheight']==100.0]

# remove 80% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=13.16]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=11.02]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=1.61]

# remove 60% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=7.49]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=6.28]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=0.92]

# remove 40% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=4.18]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=3.5]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=0.51]

# remove 20% shielding
source_dataset = source_dataset[source_dataset['shieldingdensity']!=1.82]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=1.53]
source_dataset = source_dataset[source_dataset['shieldingdensity']!=0.22]


#### Remove empty spectra

In [8]:
zero_count_indicies = np.argwhere(np.sum(source_dataset.values[:,6:],axis=1) == 0).flatten()

print('indicies dropped: ' +str(zero_count_indicies))

source_dataset.drop(source_dataset.index[zero_count_indicies], inplace=True)

indicies dropped: []


#### Add empty spectra for background 

In [9]:
blank_spectra = []
for fwhm in set(source_dataset['fwhm']):
    num_examples = source_dataset[(source_dataset['fwhm']==fwhm) &
                                  (source_dataset['isotope']==source_dataset['isotope'].iloc()[0])].shape[0]
    for k in range(num_examples):
        blank_spectra_tmp = [0]*1200
        blank_spectra_tmp[5] = fwhm
        blank_spectra_tmp[0] = 'background'
        blank_spectra_tmp[3] = 'background'
        blank_spectra.append(blank_spectra_tmp)

source_dataset = source_dataset.append(pd.DataFrame(blank_spectra,
                                                    columns=source_dataset.columns))

#### Create dataset from spectra

In [10]:
spectra_dataset = source_dataset.values[:,5:].astype('float64')
all_keys = source_dataset['isotope'].values

## Model Construction

In [11]:
model_id='DNN-kfoldseasy-1_15_dae_features'
with open('../hyperparameter_search/hyperparameter-search-results/'+model_id,"rb" ) as f:
        model_features=pickle.load(f)
        
model_class = DNN

## Define Training Parameters

#### Define online data augmentation

In [12]:
def integration_time():
    return np.random.uniform(np.log10(60),np.log10(600))

def background_cps():
    return np.random.poisson(200)

def signal_to_background():
    return np.random.uniform(0.5,2)

def calibration():
    return [np.random.uniform(0,10),
            np.random.uniform(2800/3000,3200/3000),
            0]

online_data_augmentation = online_data_augmentation_vanilla(background_dataset,
                                background_cps,
                                integration_time,
                                signal_to_background,
                                calibration,)

#### Create temporary testing dataset based on training dataset 

In [14]:
examples_per_isotope=10

testing_spectra = []
testing_keys = []

for key in set(all_keys):
    for _ in range(examples_per_isotope):
        testing_spectra_tmp = source_dataset[source_dataset['isotope']==key].sample().values[:,5:].astype('float64')
        testing_spectra_tmp = online_data_augmentation(testing_spectra_tmp).numpy()[0]
        testing_keys.append(key)
        testing_spectra.append(testing_spectra_tmp)

testing_spectra = np.array(testing_spectra)        

## Train network

In [15]:
model_id='DNN_onlinedataaugeasy_log10integrationtime'

In [ ]:
mlb=LabelBinarizer()

training_errors = []
total_networks = 10
for network_id in range(total_networks):
    # reset model on each iteration
    model = model_class(model_features)
    optimizer = tf.train.AdamOptimizer(model_features.learining_rate)
    all_keys_binarized = mlb.fit_transform(all_keys.reshape([all_keys.shape[0],1]))
    testing_keys_binarized = mlb.transform(testing_keys)
    _, f1_error = model.fit_batch(
        (spectra_dataset, all_keys_binarized),
        (testing_spectra, testing_keys_binarized),
        optimizer=optimizer,
        num_epochs=1500,
        verbose=10,
        obj_cost=model.cross_entropy,
        earlystop_cost_fn=model.f1_error,
        earlystop_patience=1000,
        data_augmentation=online_data_augmentation,
        augment_testing_data=False,
        print_errors=True,
        record_train_errors=False)

    training_errors.append(f1_error['test'])
    np.save('./final-models/final_test_errors_'+model_id, training_errors)
    model.save_weights('./final-models/'+model_id+'_checkpoint_'+str(network_id))


Epoch 10: CostFunc loss: 0.00 10.66, EarlyStop loss: 0.00 0.85
Epoch 20: CostFunc loss: 0.00 8.79, EarlyStop loss: 0.00 0.74
Epoch 30: CostFunc loss: 0.00 7.47, EarlyStop loss: 0.00 0.66
Epoch 40: CostFunc loss: 0.00 6.30, EarlyStop loss: 0.00 0.61
Epoch 50: CostFunc loss: 0.00 5.50, EarlyStop loss: 0.00 0.54
Epoch 60: CostFunc loss: 0.00 4.88, EarlyStop loss: 0.00 0.46
Epoch 70: CostFunc loss: 0.00 4.34, EarlyStop loss: 0.00 0.45
Epoch 80: CostFunc loss: 0.00 3.97, EarlyStop loss: 0.00 0.36
Epoch 90: CostFunc loss: 0.00 3.74, EarlyStop loss: 0.00 0.43
Epoch 100: CostFunc loss: 0.00 3.44, EarlyStop loss: 0.00 0.38
Epoch 110: CostFunc loss: 0.00 3.21, EarlyStop loss: 0.00 0.34
Epoch 120: CostFunc loss: 0.00 3.07, EarlyStop loss: 0.00 0.37
Epoch 130: CostFunc loss: 0.00 2.87, EarlyStop loss: 0.00 0.29
Epoch 140: CostFunc loss: 0.00 2.72, EarlyStop loss: 0.00 0.30
Epoch 150: CostFunc loss: 0.00 2.66, EarlyStop loss: 0.00 0.27
Epoch 160: CostFunc loss: 0.00 2.49, EarlyStop loss: 0.00 0.26


Epoch 1310: CostFunc loss: 0.00 0.96, EarlyStop loss: 0.00 0.13
Epoch 1320: CostFunc loss: 0.00 0.95, EarlyStop loss: 0.00 0.14
Epoch 1330: CostFunc loss: 0.00 1.00, EarlyStop loss: 0.00 0.16
Epoch 1340: CostFunc loss: 0.00 0.92, EarlyStop loss: 0.00 0.11
Epoch 1350: CostFunc loss: 0.00 0.96, EarlyStop loss: 0.00 0.13
Epoch 1360: CostFunc loss: 0.00 0.99, EarlyStop loss: 0.00 0.17
Epoch 1370: CostFunc loss: 0.00 1.02, EarlyStop loss: 0.00 0.17
Epoch 1380: CostFunc loss: 0.00 0.97, EarlyStop loss: 0.00 0.15
Epoch 1390: CostFunc loss: 0.00 0.97, EarlyStop loss: 0.00 0.17
Epoch 1400: CostFunc loss: 0.00 0.96, EarlyStop loss: 0.00 0.15
Epoch 1410: CostFunc loss: 0.00 0.99, EarlyStop loss: 0.00 0.16
Epoch 1420: CostFunc loss: 0.00 0.90, EarlyStop loss: 0.00 0.14
Epoch 1430: CostFunc loss: 0.00 0.94, EarlyStop loss: 0.00 0.14
Epoch 1440: CostFunc loss: 0.00 0.98, EarlyStop loss: 0.00 0.17
Epoch 1450: CostFunc loss: 0.00 0.94, EarlyStop loss: 0.00 0.12
Epoch 1460: CostFunc loss: 0.00 0.93, Ea

Epoch 1110: CostFunc loss: 0.00 0.99, EarlyStop loss: 0.00 0.16
Epoch 1120: CostFunc loss: 0.00 1.05, EarlyStop loss: 0.00 0.16
Epoch 1130: CostFunc loss: 0.00 1.06, EarlyStop loss: 0.00 0.17
Epoch 1140: CostFunc loss: 0.00 0.98, EarlyStop loss: 0.00 0.14
Epoch 1150: CostFunc loss: 0.00 1.01, EarlyStop loss: 0.00 0.15
Epoch 1160: CostFunc loss: 0.00 1.00, EarlyStop loss: 0.00 0.16
Epoch 1170: CostFunc loss: 0.00 0.99, EarlyStop loss: 0.00 0.16
Epoch 1180: CostFunc loss: 0.00 0.99, EarlyStop loss: 0.00 0.17
Epoch 1190: CostFunc loss: 0.00 0.95, EarlyStop loss: 0.00 0.13
Epoch 1200: CostFunc loss: 0.00 0.97, EarlyStop loss: 0.00 0.15
Epoch 1210: CostFunc loss: 0.00 1.01, EarlyStop loss: 0.00 0.18
Epoch 1220: CostFunc loss: 0.00 0.96, EarlyStop loss: 0.00 0.16
Epoch 1230: CostFunc loss: 0.00 1.05, EarlyStop loss: 0.00 0.19
Epoch 1240: CostFunc loss: 0.00 0.99, EarlyStop loss: 0.00 0.18
Epoch 1250: CostFunc loss: 0.00 0.96, EarlyStop loss: 0.00 0.15
Epoch 1260: CostFunc loss: 0.00 1.02, Ea

Epoch 910: CostFunc loss: 0.00 1.08, EarlyStop loss: 0.00 0.16
Epoch 920: CostFunc loss: 0.00 1.07, EarlyStop loss: 0.00 0.18
Epoch 930: CostFunc loss: 0.00 1.09, EarlyStop loss: 0.00 0.16
Epoch 940: CostFunc loss: 0.00 1.22, EarlyStop loss: 0.00 0.21
Epoch 950: CostFunc loss: 0.00 1.15, EarlyStop loss: 0.00 0.19
Epoch 960: CostFunc loss: 0.00 1.12, EarlyStop loss: 0.00 0.16
Epoch 970: CostFunc loss: 0.00 1.19, EarlyStop loss: 0.00 0.23
Epoch 980: CostFunc loss: 0.00 1.08, EarlyStop loss: 0.00 0.14
Epoch 990: CostFunc loss: 0.00 1.10, EarlyStop loss: 0.00 0.15
Epoch 1000: CostFunc loss: 0.00 1.03, EarlyStop loss: 0.00 0.16
Epoch 1010: CostFunc loss: 0.00 1.03, EarlyStop loss: 0.00 0.15
Epoch 1020: CostFunc loss: 0.00 1.08, EarlyStop loss: 0.00 0.16
Epoch 1030: CostFunc loss: 0.00 1.03, EarlyStop loss: 0.00 0.13
Epoch 1040: CostFunc loss: 0.00 1.01, EarlyStop loss: 0.00 0.13
Epoch 1050: CostFunc loss: 0.00 0.99, EarlyStop loss: 0.00 0.14
Epoch 1060: CostFunc loss: 0.00 1.01, EarlyStop l

Epoch 710: CostFunc loss: 0.00 1.21, EarlyStop loss: 0.00 0.16
Epoch 720: CostFunc loss: 0.00 1.25, EarlyStop loss: 0.00 0.17
Epoch 730: CostFunc loss: 0.00 1.30, EarlyStop loss: 0.00 0.18
Epoch 740: CostFunc loss: 0.00 1.27, EarlyStop loss: 0.00 0.19
Epoch 750: CostFunc loss: 0.00 1.24, EarlyStop loss: 0.00 0.17
Epoch 760: CostFunc loss: 0.00 1.32, EarlyStop loss: 0.00 0.23
Epoch 770: CostFunc loss: 0.00 1.24, EarlyStop loss: 0.00 0.15
Epoch 780: CostFunc loss: 0.00 1.28, EarlyStop loss: 0.00 0.20
Epoch 790: CostFunc loss: 0.00 1.25, EarlyStop loss: 0.00 0.18
Epoch 800: CostFunc loss: 0.00 1.22, EarlyStop loss: 0.00 0.16
Epoch 810: CostFunc loss: 0.00 1.18, EarlyStop loss: 0.00 0.16
Epoch 820: CostFunc loss: 0.00 1.23, EarlyStop loss: 0.00 0.19
Epoch 830: CostFunc loss: 0.00 1.18, EarlyStop loss: 0.00 0.18
Epoch 840: CostFunc loss: 0.00 1.16, EarlyStop loss: 0.00 0.14
Epoch 850: CostFunc loss: 0.00 1.12, EarlyStop loss: 0.00 0.14
Epoch 860: CostFunc loss: 0.00 1.11, EarlyStop loss: 0.

...

Epoch 110: CostFunc loss: 0.00 3.13, EarlyStop loss: 0.00 0.33
Epoch 120: CostFunc loss: 0.00 3.02, EarlyStop loss: 0.00 0.32
Epoch 130: CostFunc loss: 0.00 2.81, EarlyStop loss: 0.00 0.27
Epoch 140: CostFunc loss: 0.00 2.68, EarlyStop loss: 0.00 0.31
Epoch 150: CostFunc loss: 0.00 2.59, EarlyStop loss: 0.00 0.30
Epoch 160: CostFunc loss: 0.00 2.51, EarlyStop loss: 0.00 0.31
Epoch 170: CostFunc loss: 0.00 2.40, EarlyStop loss: 0.00 0.28
Epoch 180: CostFunc loss: 0.00 2.27, EarlyStop loss: 0.00 0.26
Epoch 190: CostFunc loss: 0.00 2.24, EarlyStop loss: 0.00 0.28
Epoch 200: CostFunc loss: 0.00 2.20, EarlyStop loss: 0.00 0.24
Epoch 210: CostFunc loss: 0.00 2.13, EarlyStop loss: 0.00 0.27
Epoch 220: CostFunc loss: 0.00 2.07, EarlyStop loss: 0.00 0.24
Epoch 230: CostFunc loss: 0.00 2.11, EarlyStop loss: 0.00 0.26
Epoch 240: CostFunc loss: 0.00 2.09, EarlyStop loss: 0.00 0.28
Epoch 250: CostFunc loss: 0.00 1.94, EarlyStop loss: 0.00 0.26
Epoch 260: CostFunc loss: 0.00 1.88, EarlyStop loss: 0.

Epoch 1410: CostFunc loss: 0.00 0.93, EarlyStop loss: 0.00 0.15
Epoch 1420: CostFunc loss: 0.00 0.91, EarlyStop loss: 0.00 0.13
Epoch 1430: CostFunc loss: 0.00 0.97, EarlyStop loss: 0.00 0.14
Epoch 1440: CostFunc loss: 0.00 0.92, EarlyStop loss: 0.00 0.14
Epoch 1450: CostFunc loss: 0.00 0.95, EarlyStop loss: 0.00 0.14
Epoch 1460: CostFunc loss: 0.00 0.93, EarlyStop loss: 0.00 0.14
Epoch 1470: CostFunc loss: 0.00 0.90, EarlyStop loss: 0.00 0.14
Epoch 1480: CostFunc loss: 0.00 0.91, EarlyStop loss: 0.00 0.13
Epoch 1490: CostFunc loss: 0.00 0.96, EarlyStop loss: 0.00 0.15
Epoch 1500: CostFunc loss: 0.00 0.93, EarlyStop loss: 0.00 0.15
Epoch 10: CostFunc loss: 0.00 10.73, EarlyStop loss: 0.00 0.91
Epoch 20: CostFunc loss: 0.00 9.00, EarlyStop loss: 0.00 0.85
Epoch 30: CostFunc loss: 0.00 7.55, EarlyStop loss: 0.00 0.70
Epoch 40: CostFunc loss: 0.00 6.43, EarlyStop loss: 0.00 0.60
Epoch 50: CostFunc loss: 0.00 5.53, EarlyStop loss: 0.00 0.55
Epoch 60: CostFunc loss: 0.00 4.84, EarlyStop los

Epoch 1210: CostFunc loss: 0.00 1.02, EarlyStop loss: 0.00 0.17
Epoch 1220: CostFunc loss: 0.00 1.14, EarlyStop loss: 0.00 0.22
Epoch 1230: CostFunc loss: 0.00 1.12, EarlyStop loss: 0.00 0.20
Epoch 1240: CostFunc loss: 0.00 0.97, EarlyStop loss: 0.00 0.14
Epoch 1250: CostFunc loss: 0.00 1.01, EarlyStop loss: 0.00 0.15
Epoch 1260: CostFunc loss: 0.00 1.08, EarlyStop loss: 0.00 0.19
Epoch 1270: CostFunc loss: 0.00 0.99, EarlyStop loss: 0.00 0.15
Epoch 1280: CostFunc loss: 0.00 1.03, EarlyStop loss: 0.00 0.17
Epoch 1290: CostFunc loss: 0.00 0.95, EarlyStop loss: 0.00 0.14
Epoch 1300: CostFunc loss: 0.00 0.95, EarlyStop loss: 0.00 0.14
Epoch 1310: CostFunc loss: 0.00 0.96, EarlyStop loss: 0.00 0.14
Epoch 1320: CostFunc loss: 0.00 0.92, EarlyStop loss: 0.00 0.12
Epoch 1330: CostFunc loss: 0.00 0.90, EarlyStop loss: 0.00 0.13
Epoch 1340: CostFunc loss: 0.00 0.91, EarlyStop loss: 0.00 0.12
Epoch 1350: CostFunc loss: 0.00 0.94, EarlyStop loss: 0.00 0.14
Epoch 1360: CostFunc loss: 0.00 0.93, Ea

Epoch 1010: CostFunc loss: 0.00 1.19, EarlyStop loss: 0.00 0.21
Epoch 1020: CostFunc loss: 0.00 1.10, EarlyStop loss: 0.00 0.16
Epoch 1030: CostFunc loss: 0.00 1.08, EarlyStop loss: 0.00 0.17
Epoch 1040: CostFunc loss: 0.00 1.11, EarlyStop loss: 0.00 0.18
Epoch 1050: CostFunc loss: 0.00 1.07, EarlyStop loss: 0.00 0.16
Epoch 1060: CostFunc loss: 0.00 1.09, EarlyStop loss: 0.00 0.18
Epoch 1070: CostFunc loss: 0.00 1.05, EarlyStop loss: 0.00 0.14
Epoch 1080: CostFunc loss: 0.00 1.04, EarlyStop loss: 0.00 0.17
Epoch 1090: CostFunc loss: 0.00 1.05, EarlyStop loss: 0.00 0.17
Epoch 1100: CostFunc loss: 0.00 1.01, EarlyStop loss: 0.00 0.14
Epoch 1110: CostFunc loss: 0.00 1.04, EarlyStop loss: 0.00 0.15
Epoch 1120: CostFunc loss: 0.00 0.99, EarlyStop loss: 0.00 0.13
Epoch 1130: CostFunc loss: 0.00 1.02, EarlyStop loss: 0.00 0.14
Epoch 1140: CostFunc loss: 0.00 1.01, EarlyStop loss: 0.00 0.14
Epoch 1150: CostFunc loss: 0.00 1.04, EarlyStop loss: 0.00 0.15
Epoch 1160: CostFunc loss: 0.00 1.10, Ea

Epoch 810: CostFunc loss: 0.00 1.12, EarlyStop loss: 0.00 0.14
Epoch 820: CostFunc loss: 0.00 1.10, EarlyStop loss: 0.00 0.14
Epoch 830: CostFunc loss: 0.00 1.14, EarlyStop loss: 0.00 0.16
Epoch 840: CostFunc loss: 0.00 1.23, EarlyStop loss: 0.00 0.21
Epoch 850: CostFunc loss: 0.00 1.15, EarlyStop loss: 0.00 0.16
Epoch 860: CostFunc loss: 0.00 1.15, EarlyStop loss: 0.00 0.17
Epoch 870: CostFunc loss: 0.00 1.15, EarlyStop loss: 0.00 0.18
Epoch 880: CostFunc loss: 0.00 1.15, EarlyStop loss: 0.00 0.16
Epoch 890: CostFunc loss: 0.00 1.12, EarlyStop loss: 0.00 0.14
Epoch 900: CostFunc loss: 0.00 1.12, EarlyStop loss: 0.00 0.16
Epoch 910: CostFunc loss: 0.00 1.17, EarlyStop loss: 0.00 0.17
Epoch 920: CostFunc loss: 0.00 1.18, EarlyStop loss: 0.00 0.17
Epoch 930: CostFunc loss: 0.00 1.11, EarlyStop loss: 0.00 0.13
Epoch 940: CostFunc loss: 0.00 1.20, EarlyStop loss: 0.00 0.18
Epoch 950: CostFunc loss: 0.00 1.06, EarlyStop loss: 0.00 0.14
Epoch 960: CostFunc loss: 0.00 1.11, EarlyStop loss: 0.